In [ ]:
#Load and install dependencies
!pip install mistralai
!pip install python-dotenv
import os
from PIL import Image
import base64
import matplotlib.pyplot as plt
from mistralai import Mistral
from dotenv import load_dotenv
load_dotenv()

In [ ]:
#Set API key and select model
api_key = "" #mistral api key. replace with your own key


model = "pixtral-12b-2409" #replace with finetuned model
client = Mistral(api_key=api_key) #mistral api key


In [ ]:
#function to encode local images
def encode_image_base64(image_path):
 with open(image_path, "rb") as image_file:
   return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:
#load in directory
from google.colab import drive
drive.mount('/content/drive')

!ls /content/drive/MyDrive/clarivex/jpgs/ #replace with file path to your images, either locally or what have you

In [ ]:
#loop to load in images
input_folder = '/content/drive/MyDrive/clarivex/jpgs/'
directory = input_folder
files = os.listdir(directory)

#then encode into readable format for Pixtral and append to a new list of images
images = []
for i in files:
    file_path = os.path.join(directory, i)  # Get full path of the image file
    images.append(encode_image_base64(file_path))

In [ ]:
#extract zero shot labels from diagnostic report

def zero_shot_labels(diagnostic_report):
  diagnostic_report = diagnostic_report.lower()  # Ensure case-insensitive matching

  if "malignant" in diagnostic_report:
      return "malignant"
  elif "benign" in diagnostic_report:
      return "benign"
  else:
      return "unknown"

In [ ]:
# Empty list to store results
zero_shot_preds = []

# Loop through each image and obtain zero shot descriptions
for image in images:
    chat_response = client.chat.complete(
        model=model,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Provide a clinical analysis of the image describing colour, texture, shape, size, and location, as well as clearly stating if you believe it to be malignant or benign."
                    },
                    {
                        "type": "image_url",
                        "image_url": f"data:image/jpeg;base64,{image}"
                    }
                ]
            },
        ]
    )

    # Get the diagnostic report from the response
    diagnostic_report = chat_response.choices[0].message.content

    # Extract whether the tumor is benign or malignant
    zero_shot_prediction = zero_shot_labels(diagnostic_report)

    # Append the label to the results list
    zero_shot_preds.append(zero_shot_prediction)

In [ ]:
len(zero_shot_preds)


In [ ]:
import pandas as pd

# Load the CSV file
!ls /content/drive/MyDrive/clarivex/ddi_metadata.csv # load in ground truth labels
file_path = '/content/drive/MyDrive/clarivex/ddi_metadata.csv'
df = pd.read_csv(file_path)

# Extract relevant rows of the 'malignant' column
malignant_data = df['malignant'][:len(zero_shot_preds)]

# Relabel 'True' as 'malignant' and 'False' as 'benign'
relabelled_data = malignant_data.apply(lambda x: 'malignant' if x else 'benign')

# Convert to list
ground_labels = relabelled_data.tolist()

In [ ]:
#function to compute accuracy of zero shot predictions

def zero_shot_accuracy(output_labels, ground_labels):
  if len(output_labels) != len(ground_labels):
    raise ValueError("Length of predicted and ground truth labels must be the same.")

  # Count correct predictions
  correct_predictions = sum([pred == actual for pred, actual in zip(output_labels, ground_labels)])

  # Calculate accuracy
  accuracy = correct_predictions / len(ground_labels)

  return accuracy

In [ ]:
# compute accuracy
accuracy = zero_shot_accuracy(zero_shot_preds, ground_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")